In [6]:
import logging
import time
from concurrent.futures import ThreadPoolExecutor

import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [7]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [48]:
lenta_blocs_indexes = [1,
                       3,
                       4,
                       5,
                       8,
                       37,
                       48,
                       87]

In [18]:
from abc import ABC, abstractmethod
from typing import TypedDict, NotRequired


class ParamsDictType(TypedDict):
    size: int
    search_from: int
    bloc: NotRequired[int]
    query: NotRequired[str]


class NewsParser(ABC):
    @abstractmethod
    def parse_category(self, bloc: int = None, query: str = None) -> pd.DataFrame:
        pass

    @abstractmethod
    def parse(self) -> pd.DataFrame:
        pass

    @abstractmethod
    def _get_page_data(self, url: str) -> pd.DataFrame:
        pass


In [49]:
class LentaParser(NewsParser):
    @staticmethod
    def _get_url(params: ParamsDictType) -> str:
        base_url = (
            f"https://lenta.ru/search/v2/process?size={params['size']}&from={params['search_from']}"
            f"&sort=2&domain=1&modified,format=yyyy-MM-dd&modified,from=2023-01-01&type=1"
        )

        if 'query' in params and params['query']:
            base_url += f"&query={params['query']}"
        if 'bloc' in params and params['bloc']:
            base_url += f"&bloc={params['bloc']}"

        return base_url

    def _get_page_data(self, url: str) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей по текущему url
        """
        try:
            time.sleep(2)
            response = rq.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()
            if 'matches' in data:
                return pd.DataFrame(data['matches'])

            logging.warning(f"Статьи не найдены: {url}")
            return pd.DataFrame()
        except Exception as e:
            logging.error(f"Ошибка при загрузке url {url}: {e}")
            return pd.DataFrame()

    def parse_category(self, bloc: int = None, query: str = None) -> pd.DataFrame:
        """
        Парсит переданную категорию и возвращает pd.DataFrame
        """
        size = 250  # скачиванием батчами, чтобы не вылететь по таймауту
        search_from = 1
        articles_df = pd.DataFrame()

        while search_from < 4000:
            params = {
                'size': size,
                'search_from': search_from,
            }
            if query:
                params['query'] = query
            if bloc:
                params['bloc'] = bloc

            url = self._get_url(params)
            page_data = self._get_page_data(url)

            if page_data.empty:
                logging.info(f"Не найдено данных для категории {bloc}")
                break

            articles_df = pd.concat([articles_df, page_data], ignore_index=True)
            search_from += size

        logging.info(f"Для категории {bloc} найдено {articles_df.shape[0]} статей")
        return articles_df

    def parse(self) -> pd.DataFrame:
        """
        Парсит все переданные категории и возвращает обший pd.DataFrame
        """
        df = pd.DataFrame()

        with ThreadPoolExecutor() as executor:
            results = list(executor.map(self.parse_category, lenta_blocs_indexes))

        for result in results:
            df = pd.concat([df, result], ignore_index=True)

        query_results = self.parse_category(query='Строительство')
        query_results['bloc'] = 6
        df = pd.concat([df, query_results], ignore_index=True)

        logging.info(f"Всего статей: {df.shape[0]}")
        return df

In [63]:
lenta_parser = LentaParser()
lenta_df = lenta_parser.parse()
lenta_df.shape

2024-12-22 14:05:26,231 - INFO - Для категории 37 найдено 4000 статей
2024-12-22 14:05:27,184 - INFO - Для категории 48 найдено 4000 статей
2024-12-22 14:05:28,140 - INFO - Для категории 3 найдено 4000 статей
2024-12-22 14:05:31,376 - INFO - Для категории 5 найдено 4000 статей
2024-12-22 14:05:31,525 - INFO - Для категории 8 найдено 4000 статей
2024-12-22 14:05:32,059 - INFO - Для категории 4 найдено 4000 статей
2024-12-22 14:05:33,103 - INFO - Для категории 1 найдено 4000 статей
2024-12-22 14:05:34,192 - INFO - Для категории 87 найдено 4000 статей
2024-12-22 14:06:45,942 - INFO - Для категории None найдено 4000 статей
2024-12-22 14:06:45,950 - INFO - Всего статей: 36000


(36000, 16)

In [64]:
lenta_df.head()

,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,1753996,https://lenta.ru/news/2024/12/22/putin-predupr...,Путин предупредил об ответе России на эскалацию,1734864598,1734864598,1,1,0,0,1,[1],https://icdn.lenta.ru/images/2024/12/22/13/202...,1734864598,Фото: Гавриил Григоров / РИА Новости Фариза Ба...,Путин предупредил об ответе России на эскалацию,Фото: Гавриил Григоров / РИА Новости ... ситуа...
1,1753995,https://lenta.ru/news/2024/12/22/v-kurskoy-obl...,В Курской области отражены три атаки украински...,1734864484,1734864530,1,1,0,0,1,[1],https://icdn.lenta.ru/images/2024/12/22/13/202...,1734864484,Фото: Сергей Бобылев / РИА Новости Алан Босико...,В Курской области отражены три атаки украински...,Фото: Сергей Бобылев / РИА Новости Алан ... (В...
2,1753991,https://lenta.ru/news/2024/12/22/v-kazani-pozh...,В Казани пожаловались на массовую недоступност...,1734863340,1734864082,1,1,0,0,1,[2],https://icdn.lenta.ru/images/2024/12/22/13/202...,1734863340,Фото: Павел Колядин / РИА Новости Алан Босиков...,В Казани пожаловались на массовую недоступност...,Фото: Павел Колядин / РИА Новости Алан ... лет...
3,1753984,https://lenta.ru/news/2024/12/22/peskov-zayavi...,Песков заявил об искренне поразившем его блоге...,1734862140,1734864181,1,1,0,0,1,[2],https://icdn.lenta.ru/images/2024/12/22/13/202...,1734862140,Фото: Дмитрий Азаров / Коммерсантъ Фариза Баца...,Песков заявил об искренне поразившем его блоге...,Фото: Дмитрий Азаров / Коммерсантъ ... на прям...
4,1753985,https://lenta.ru/news/2024/12/22/putin-rasskaz...,Путин рассказал о спорах в Минобороны вокруг «...,1734861780,1734862979,1,1,0,0,1,[1],https://icdn.lenta.ru/images/2024/12/22/13/202...,1734861780,Фото: Министерство обороны РФ / РИА Новости Ал...,Путин рассказал о спорах в Минобороны вокруг «...,Фото: Министерство обороны РФ / РИА ... россий...


In [65]:
lenta_df['bloc'].value_counts(normalize=True)

bloc
1     0.111111
3     0.111111
4     0.111111
5     0.111111
8     0.111111
37    0.111111
48    0.111111
87    0.111111
6     0.111111
Name: proportion, dtype: float64

In [66]:
df = lenta_df[['bloc', 'text']]
df = df[df['text'].notnull() & (df['text'].str.split().str.len() > 5)]
df = df.drop_duplicates()

df.shape

(36000, 2)

In [67]:
lenta_categories = {
    1: 'Общество/Россия',
    3: 'Бывший СССР',
    4: 'Экономика',
    5: 'Наука и техника',
    8: 'Спорт',
    37: 'Силовые структуры',
    48: 'Туризм/Путешествия',
    87: 'Забота о себе',
    6: 'Строительство'
}
article_categories = {
    0: 'Общество/Россия',
    1: 'Экономика',
    2: 'Силовые структуры',
    3: 'Бывший СССР',
    4: 'Спорт',
    5: 'Забота о себе',
    6: 'Строительство',
    7: 'Туризм/Путешествия',
    8: 'Наука и техника'
}

new_mapping = {v: k for k, v in article_categories.items()}
df['bloc'] = df['bloc'].dropna().map(lenta_categories).map(new_mapping)
df['bloc'].value_counts(normalize=True)

bloc
0    0.111111
3    0.111111
1    0.111111
8    0.111111
4    0.111111
2    0.111111
7    0.111111
5    0.111111
6    0.111111
Name: proportion, dtype: float64

In [68]:
df.to_csv('./lenta_df.csv')